In [30]:
from os import listdir
from os.path import isfile, join

import pandas as pd
import re

cat_cols = [ # These are binary
    "bare_soil"
    , "annual_cover_crop"
    , "permanent_cover_crop_native"
    , "permanent_cover_crop_non_native"
    , "permanent_cover_crop_volunteer_sward"
    , "irrigation_energy_diesel"
    , "irrigation_energy_electricity"
    , "irrigation_energy_pressure"
    , "irrigation_energy_solar"
    , "irrigation_type_dripper"
    , "irrigation_type_flood"
    , "irrigation_type_non_irrigated"
    , "irrigation_type_overhead_sprinkler"
    , "irrigation_type_undervine_sprinkler"
    , 'river_water'
    , 'groundwater'
    , 'surface_water_dam'
    , 'recycled_water_from_other_source'
    , 'mains_water'
    , 'other_water'
    , 'water_applied_for_frost_control'
    , "nh_frost"
    , "nh_disease"
    , "data_year_id" # These are one hot encoded
    , "giregion"
]

files = [f for f in listdir("./") if isfile(join("./", f))]

r = re.compile(".*loss")
files = list(filter(r.match, files))

df = pd.read_csv(files[0], index_col=0, names=[files[0][:-9]], header=0)

for filename in files[1:]:
    df[filename[:-9]] = pd.read_csv(files[0], index_col=0, header=0)

for col in list(set(cat_cols) - set(df.index)):
    r = re.compile("{}.*".format(col))
    rows = list(filter(r.match, list(df.index)))
    df.loc[col] = df.loc[rows].mean()
    df = df.drop(rows).copy()

df = df.stack().reset_index()
df.columns = ["source", "links", "value"]


In [36]:
df["value"] = round(df["value"]*1000000000).apply(int)
df

source  \
0           tonnes_grapes_harvested   
1           tonnes_grapes_harvested   
2           tonnes_grapes_harvested   
3           tonnes_grapes_harvested   
4           tonnes_grapes_harvested   
...                             ...   
1975  irrigation_energy_electricity   
1976  irrigation_energy_electricity   
1977  irrigation_energy_electricity   
1978  irrigation_energy_electricity   
1979  irrigation_energy_electricity   

                                                  links  value  
0                                            nh_disease      0  
1                            irrigation_energy_pressure      0  
2                                           mains_water      0  
3     fungicide_spraying_number_of_times_passes_per_...      0  
4                                        area_harvested      0  
...                                                 ...    ...  
1975               permanent_cover_crop_volunteer_sward      0  
1976                                         water_used      0  
1977                                 biodiesel_vineyard      0  
1978                      irrigation_energy_electricity      0  
1979                                    petrol_vineyard      0  

[1980 rows x 3 columns]

In [37]:
hv.Chord(df)

:Chord   [source,links]   (value)

In [38]:
chord = hv.Chord(df)
chord.opts(
    opts.Chord(cmap='Category20', edge_cmap='Category20', edge_color=dim('source').str(), 
               labels='index', node_color=dim('index').str()))

:Chord   [source,links]   (value)